# Battle of the Neighborhoods in Vilnius – the Case of New Gas Station

## Business case

A company that owns a network of gas stations wants to expand and build a new gas station in the city of Vilnius. 
The problem is - how to find the best place for their new gas station in Vilnius. First they need to decide what neighborhood in Vilnius would be most suitable for building a new gas station. 

There are 21 neighborhoods in the city of Vilnius. 
In order to make the right decision they have to examine how many gas stations there are already in each neighborhood, what is the population in each neighborhood and what is the traffic volume in each neighborhood. 
These are the main aspects to which they must pay attention when choosing a neighborhood. 
You should choose to build a new gas station in place where the traffic volume is the highest and where number of gas stations per resident is the lowest.

## Data requirements and analytical approach

In order to solve this problem and help company to decide in which neighborhood they should build a new gas station, we need this data:

•	Data of gas stations in the city of Vilnius – how many in each neighborhood. Gas stations location data we will get from Fourthsquare.

•	Data of neighborhoods in Vilnius – number and names of neighborhoods, the coordinates of each neighborhood. There are 21 neighborhood in Vilnius, the names and coordinates we can find in Wikipedia page (https://lt.wikipedia.org/wiki/Vilnius).

•	Data of the residents in Vilnius – number of residents in each neighborhood, population data we can find in Wikipedia page (https://lt.wikipedia.org/wiki/Vilnius). Number of the residents in each neighborhood will be used to calculate the number of gas stations per resident.

•	Data about traffic volume in each neighborhood In Vilnius – there are 245 traffic junctions in Vilnius where the traffic volume is measured: how many cars have crossed the junction in one hour, average of cars that have crossed the junction in particular month and so on (open data in Github https://github.com/vilnius/traffic). And 50 traffic junctions are with most intense traffic volume. The data of these 50 traffic junctions will be used to identify which neighborhood has the highest traffic volume. This data is stored there https://github.com/akisieliene/Coursera_Capstone/blob/master/traffic.csv. We will calculate the number of gas stations per one traffic junctions.

Having all this data about each neighborhood in Vilnius – number of gas stations per resident and traffic volume – we will be able to group and compare neighborhoods in Vilnius and find the most suitable neighborhood for a new gas station. Company should choose to build a new gas station in place where the traffic volume is the highest and where number of gas stations per resident is the lowest. In order to group neighborhoods we will use above data and run kmeans clustering algorithm.

## Data collection and preparation

#### Importing all the necessery libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means for clustering stage
from sklearn.cluster import KMeans

# import metrics for model evaluation
from sklearn import metrics
from sklearn.metrics import davies_bouldin_score

# import StandardScaler for data normalization
from sklearn.preprocessing import StandardScaler

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

# import BeautifulSoup for scraping the webpage information
from bs4 import BeautifulSoup

import csv # library to handle csv files

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

#### Getting the Neighborhoods data from Wikipedia

First we need data of neighborhoods in Vilnius. There are 21 neighborhood in Vilnius. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains all the neighborhoods that exist as well as the latitude and logitude coordinates of each neighborhood.

The data of Vilnius neighborhoods we can find in Wikipedia page https://lt.wikipedia.org/wiki/S%C4%85ra%C5%A1as:Vilniaus_seni%C5%ABnijos. All we need to do is to scrap this information into pandas data frame.

In [2]:
# Scraping the table data from Wikipedia
List_neigh=pd.read_html('https://en.wikipedia.org/wiki/Neighborhoods_of_Vilnius')
Neigh=pd.DataFrame(List_neigh[0])
Neigh.head()

,Unnamed: 0,Neighborhoods,Area (km2)[1],Pop. (2001)[1],Density (2001)[1],Latitude,Longitude
0,1,Verkiai,56.0,30856,551.0,54.708707,25.284686
1,2,Antakalnis,77.2,39697,514.2,54.701126,25.308957
2,3,Pašilaičiai,7.9,25674,3249.9,54.725942,25.231328
3,4,Fabijoniškės,5.9,36644,6210.8,54.723397,25.249529
4,5,Pilaitė,13.9,15996,1150.8,54.708126,25.175803


Now we will drop the unnecessary columns and rename columns

In [3]:
# Dropping unnecessary columns
Neigh.drop(["Unnamed: 0", "Area (km2)[1]", "Density (2001)[1]"], axis=1, inplace=True)
Neigh.head()

,Neighborhoods,Pop. (2001)[1],Latitude,Longitude
0,Verkiai,30856,54.708707,25.284686
1,Antakalnis,39697,54.701126,25.308957
2,Pašilaičiai,25674,54.725942,25.231328
3,Fabijoniškės,36644,54.723397,25.249529
4,Pilaitė,15996,54.708126,25.175803


In [4]:
# Renaming column
Neigh.rename(columns={"Pop. (2001)[1]":"Population"}, inplace=True)
Neigh.head(21)

,Neighborhoods,Population,Latitude,Longitude
0,Verkiai,30856,54.708707,25.284686
1,Antakalnis,39697,54.701126,25.308957
2,Pašilaičiai,25674,54.725942,25.231328
3,Fabijoniškės,36644,54.723397,25.249529
4,Pilaitė,15996,54.708126,25.175803
5,Justiniškės,30958,54.717905,25.220236
6,Viršuliškės,16250,54.717867,25.220222
7,Šeškinė,36604,54.715694,25.244574
8,Šnipiškės,19321,54.692956,25.285007
9,Žirmūnai,47410,54.723249,25.297213


#### Use geopy library to get the latitude and longitude values of Vilnius City.

In [5]:
address = 'Vilnius'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Vilnius are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Vilnius are 54.6870458, 25.2829111.


#### Create a map of Vilnius with neighborhoods superimposed on top.

In [6]:
# create map of Vilnius using latitude and longitude values
map_vilnius = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, neighborhood in zip(Neigh['Latitude'], Neigh['Longitude'], Neigh['Neighborhoods']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_vilnius)  
    
map_vilnius

#### Getting the gas stations data from Fourthsquare

Define Foursquare Credentials and Version

In [7]:
CLIENT_ID = 'XUM1PGE3J5FQ4CE3BLXBR14ICHI10SJXL5MH5WKYTEHV2TZZ' # your Foursquare ID
CLIENT_SECRET = 'D0BAIFT2S5E2RQJVSJVJ20ZVOXWGGC2X4SGXWMOOHGQRW1IB' # your Foursquare Secret
VERSION = '20190107' # Foursquare API version
LIMIT=500
radius=2000
category_id='4bf58dd8d48988d113951735'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XUM1PGE3J5FQ4CE3BLXBR14ICHI10SJXL5MH5WKYTEHV2TZZ
CLIENT_SECRET:D0BAIFT2S5E2RQJVSJVJ20ZVOXWGGC2X4SGXWMOOHGQRW1IB


Let's create a function to find all the gas stations in all the neighborhoods in Vilnius

In [8]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            category_id, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['venues']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['name'], 
            v['location']['lat'], 
            v['location']['lng'],
            v['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',
                  'Venue Category']
    
    return(nearby_venues)

Now we create a new dataframe called *Vilnius_venues*.

In [9]:
Vilnius_venues = getNearbyVenues(names=Neigh['Neighborhoods'],
                                   latitudes=Neigh['Latitude'],
                                   longitudes=Neigh['Longitude']
                                  )


Verkiai
Antakalnis
Pašilaičiai
Fabijoniškės
Pilaitė
Justiniškės
Viršuliškės
Šeškinė
Šnipiškės
Žirmūnai
Karoliniškės
Žvėrynas
Grigiškės
Lazdynai
Vilkpėdė
Naujamiestis
Senamiestis
Naujoji Vilnia
Paneriai
Naujininkai
Rasos


In [10]:
print(Vilnius_venues.shape)
Vilnius_venues.head()

(105, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Verkiai,54.708707,25.284686,Neste Oil,54.711976,25.292909,Gas Station
1,Verkiai,54.708707,25.284686,Baltic Petroleum,54.702801,25.288381,Gas Station
2,Verkiai,54.708707,25.284686,Statoil Verkiai 1.2.3,54.715595,25.290809,Convenience Store
3,Verkiai,54.708707,25.284686,Neste Oil Ozo,54.711962,25.270586,Gas Station
4,Verkiai,54.708707,25.284686,Lukoil,54.704228,25.266920,Gas Station


Let's check how many gas stations were returned for each neighborhood

In [11]:
Gas_stations=Vilnius_venues.groupby('Neighborhood').count()
Gas_stations

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Antakalnis,4,4,4,4,4,4
Fabijoniškės,8,8,8,8,8,8
Grigiškės,1,1,1,1,1,1
Justiniškės,4,4,4,4,4,4
Karoliniškės,2,2,2,2,2,2
Lazdynai,4,4,4,4,4,4
Naujamiestis,2,2,2,2,2,2
Naujininkai,6,6,6,6,6,6
Naujoji Vilnia,1,1,1,1,1,1


Next we will merge gas stations results to existing Vilnius neighborhoods dataframe

In [12]:
# reseting the index
Gas_stations=Gas_stations['Venue'].reset_index()

In [13]:
# renaming the columns
Gas_stations=Gas_stations.rename(columns={"Neighborhood": "Neighborhoods", "Venue": "Count_gas_stations"})
Gas_stations

,Neighborhoods,Count_gas_stations
0,Antakalnis,4
1,Fabijoniškės,8
2,Grigiškės,1
3,Justiniškės,4
4,Karoliniškės,2
5,Lazdynai,4
6,Naujamiestis,2
7,Naujininkai,6
8,Naujoji Vilnia,1
9,Paneriai,4


We will merge two dataframes - the neighborhoods dataframe with locations of neighborhoods and numebr of gas stations in neighborhoods dataframe.

In [14]:
# merging the data of Vilnius neighborhoods and gas stations
Vilnius_data = pd.merge(Neigh, Gas_stations, on='Neighborhoods')
Vilnius_data

,Neighborhoods,Population,Latitude,Longitude,Count_gas_stations
0,Verkiai,30856,54.708707,25.284686,14
1,Antakalnis,39697,54.701126,25.308957,4
2,Pašilaičiai,25674,54.725942,25.231328,8
3,Fabijoniškės,36644,54.723397,25.249529,8
4,Pilaitė,15996,54.708126,25.175803,2
5,Justiniškės,30958,54.717905,25.220236,4
6,Viršuliškės,16250,54.717867,25.220222,4
7,Šeškinė,36604,54.715694,25.244574,9
8,Šnipiškės,19321,54.692956,25.285007,3
9,Žirmūnai,47410,54.723249,25.297213,13


Now we will calculate how many gas stations per resident there are in each neighborhood. We will have to devide number of gas stations in each neighborhood by population in each neighborhood.

In [15]:
Vilnius_data['Gas_station_per_person']=Vilnius_data['Count_gas_stations']/Vilnius_data['Population']
Vilnius_data

,Neighborhoods,Population,Latitude,Longitude,Count_gas_stations,Gas_station_per_person
0,Verkiai,30856,54.708707,25.284686,14,0.000454
1,Antakalnis,39697,54.701126,25.308957,4,0.000101
2,Pašilaičiai,25674,54.725942,25.231328,8,0.000312
3,Fabijoniškės,36644,54.723397,25.249529,8,0.000218
4,Pilaitė,15996,54.708126,25.175803,2,0.000125
5,Justiniškės,30958,54.717905,25.220236,4,0.000129
6,Viršuliškės,16250,54.717867,25.220222,4,0.000246
7,Šeškinė,36604,54.715694,25.244574,9,0.000246
8,Šnipiškės,19321,54.692956,25.285007,3,0.000155
9,Žirmūnai,47410,54.723249,25.297213,13,0.000274


#### Getting the traffic volume data in each neighborhood from csv file on Github

In order to measure the traffic volume in each neighborhood we will use traffic junctions data. There are 50 traffic junctions in Vilnius where the traffic volume is very high. Traffic volume is measured by average amount of cars that have crossed the junction per day (data on Github https://github.com/akisieliene/Coursera_Capstone/blob/master/traffic.csv). This data will be used to identify which neighborhood has the highest traffic volume. Let's download traffic volume data from Github and read it into pandas dataframe.

In [16]:
traffic_volume = pd.read_csv('https://raw.githubusercontent.com/akisieliene/Coursera_Capstone/master/traffic.csv')
traffic_volume.head()

,Neighborhood,Junction,Traffic_volume
0,Pašilaičiai,UkmergėsMykoloLietuvio,39447
1,Verkiai,GeležinioVilkoMokslininkų,72528
2,Verkiai,AteitiesBaltupio,44320
3,Fabijoniškės,AteitiesDidlaukio,42617
4,Fabijoniškės,AteitiesStanevičiaus,48640


Now we will count number of junctions with high traffic volume in each neighborhood. So we will have the number of intevsive traffic junctions in each neighborhood in Vilnius.

In [17]:
traffic=traffic_volume.groupby('Neighborhood').count()
traffic

,Junction,Traffic_volume
Neighborhood,,
Antakalnis,1,1
Fabijoniškės,4,4
Justiniškės,2,2
Karoliniškės,1,1
Naujamiestis,5,5
Naujininkai,3,3
Pašilaičiai,1,1
Pilaitė,1,1
Rasos,4,4


In [18]:
# reseting the index
traffic=traffic['Traffic_volume'].reset_index()
traffic

,Neighborhood,Traffic_volume
0,Antakalnis,1
1,Fabijoniškės,4
2,Justiniškės,2
3,Karoliniškės,1
4,Naujamiestis,5
5,Naujininkai,3
6,Pašilaičiai,1
7,Pilaitė,1
8,Rasos,4
9,Senamiestis,2


In [19]:
# renaming the columns
traffic=traffic.rename(columns={"Neighborhood": "Neighborhoods", "Traffic_volume": "Count_junctions"})
traffic

,Neighborhoods,Count_junctions
0,Antakalnis,1
1,Fabijoniškės,4
2,Justiniškės,2
3,Karoliniškės,1
4,Naujamiestis,5
5,Naujininkai,3
6,Pašilaičiai,1
7,Pilaitė,1
8,Rasos,4
9,Senamiestis,2


As we can see there are 21 neighborhood in Vilniu, but only in 17 there are traffic junctions where traffic volume is very high. So we have 4 neighborhoods with no intevsive traffic junctions. We will add these neighborhoods to our dataframe with value 1 (lowest value of traffic) in order to have all neighborhoods in this dataframe.
Neighborhoods which we need to add are Paneriai, Naujoji Vilnia, Lazdynai and Grigiškės.

In [20]:
# addind missing neighborhoods data
new_data = pd.DataFrame({ 'Neighborhoods' : ['Paneriai','Naujoji Vilnia','Lazdynai','Grigiškės'],
                          'Count_junctions'  : [1,1,1,1]})
traffic=traffic.append(new_data, ignore_index=True)
traffic

,Neighborhoods,Count_junctions
0,Antakalnis,1
1,Fabijoniškės,4
2,Justiniškės,2
3,Karoliniškės,1
4,Naujamiestis,5
5,Naujininkai,3
6,Pašilaičiai,1
7,Pilaitė,1
8,Rasos,4
9,Senamiestis,2


#### Merging all Vilnius neighborhoods data into one dataframe

Next we will merge two dataframes *Vilnius_data* and *traffic* into one dataframe and call it *Vilnius_data_merged*

In [21]:
# merging two dataframes 
Vilnius_data_merged = pd.merge(Vilnius_data, traffic, on='Neighborhoods')
Vilnius_data_merged

,Neighborhoods,Population,Latitude,Longitude,Count_gas_stations,Gas_station_per_person,Count_junctions
0,Verkiai,30856,54.708707,25.284686,14,0.000454,3
1,Antakalnis,39697,54.701126,25.308957,4,0.000101,1
2,Pašilaičiai,25674,54.725942,25.231328,8,0.000312,1
3,Fabijoniškės,36644,54.723397,25.249529,8,0.000218,4
4,Pilaitė,15996,54.708126,25.175803,2,0.000125,1
5,Justiniškės,30958,54.717905,25.220236,4,0.000129,2
6,Viršuliškės,16250,54.717867,25.220222,4,0.000246,2
7,Šeškinė,36604,54.715694,25.244574,9,0.000246,4
8,Šnipiškės,19321,54.692956,25.285007,3,0.000155,6
9,Žirmūnai,47410,54.723249,25.297213,13,0.000274,3


Next we will have to count how many gas stations there are per one junction where traffic volume is high. If there are few gas stations and a lot of intensive junctions this neighborhood should be considered as a good neighborhood for new gas station.

First we will check data types in our new dataframe.

In [22]:
Vilnius_data_merged.dtypes

Neighborhoods              object
Population                  int64
Latitude                  float64
Longitude                 float64
Count_gas_stations          int64
Gas_station_per_person    float64
Count_junctions             int64
dtype: object

We will devide gas stations number in each neighborhood from junctions with high traffic volume to get the gas stations number per one high traffic volume junction.

In [23]:
Vilnius_data_merged['Gas_station_per_junction']=Vilnius_data_merged['Count_gas_stations']/Vilnius_data_merged['Count_junctions']
Vilnius_data_merged

,Neighborhoods,Population,Latitude,Longitude,Count_gas_stations,Gas_station_per_person,Count_junctions,Gas_station_per_junction
0,Verkiai,30856,54.708707,25.284686,14,0.000454,3,4.666667
1,Antakalnis,39697,54.701126,25.308957,4,0.000101,1,4.000000
2,Pašilaičiai,25674,54.725942,25.231328,8,0.000312,1,8.000000
3,Fabijoniškės,36644,54.723397,25.249529,8,0.000218,4,2.000000
4,Pilaitė,15996,54.708126,25.175803,2,0.000125,1,2.000000
5,Justiniškės,30958,54.717905,25.220236,4,0.000129,2,2.000000
6,Viršuliškės,16250,54.717867,25.220222,4,0.000246,2,2.000000
7,Šeškinė,36604,54.715694,25.244574,9,0.000246,4,2.250000
8,Šnipiškės,19321,54.692956,25.285007,3,0.000155,6,0.500000
9,Žirmūnai,47410,54.723249,25.297213,13,0.000274,3,4.333333


Now we have all the data we need to decide in which neighborhood we should build a new gas station. 
We should find the neighborhood with low number of gas stations per resident and with low number of gas stations per intensive traffic junctions.

## Modeling and evaluation

#### Before building a model we will look into data we have

We will check in which neighborhoods number of gas stations per resident is smallest.

In [24]:
Vilnius_data_merged.nsmallest(3, 'Gas_station_per_person')

,Neighborhoods,Population,Latitude,Longitude,Count_gas_stations,Gas_station_per_person,Count_junctions,Gas_station_per_junction
17,Naujoji Vilnia,32775,54.690446,25.412790,1,0.000031,1,1.0
10,Karoliniškės,31175,54.685131,25.205156,2,0.000064,1,2.0
15,Naujamiestis,27892,54.685457,25.284650,2,0.000072,5,0.4


Three neighborhoods in which number of gas stations per resident is smallest are - Naujoji Vilnia, Karoliniškės and Naujamiestis. So we could think that these three neighborhoods should be considered as a good place to build new gas station.

Next we will check in which neighborhoods number of gas stations per high traffic volume junction is smallest.

In [25]:
Vilnius_data_merged.nsmallest(3, 'Gas_station_per_junction')

,Neighborhoods,Population,Latitude,Longitude,Count_gas_stations,Gas_station_per_person,Count_junctions,Gas_station_per_junction
15,Naujamiestis,27892,54.685457,25.284650,2,0.000072,5,0.4
8,Šnipiškės,19321,54.692956,25.285007,3,0.000155,6,0.5
14,Vilkpėdė,24749,54.671995,25.243926,3,0.000121,6,0.5


Three neighborhoods in which number of gas stations per junction is smallest are - Naujamiestis, Šnipiškės and Vilkpėdė. So we could think that these three neighborhoods should be considered as a good place to build new gas station.

As we can see only one neighborhood *Naujamiestis* is between top three smallest values in both cases. We could think of this neighborhood as our first choise. But we will run clustering model to cluster Vilnius neighborhoods to see maybe there are similar neighborhoods to Naujamiestis and we will have other options.

### Clustering neighborhoods of Vilnius

We should find the neighborhood with low number of gas stations per resident and with low number of gas stations per intensive traffic junctions.

First we need to leave only two columns in our dataframe: *Gas_station_per_person* and *Gas_station_per_junction*

In [26]:
Vilnius_data_clustering = Vilnius_data_merged.drop(['Neighborhoods', 'Population','Latitude','Longitude','Count_gas_stations','Count_junctions'], 1)

Vilnius_data_clustering

,Gas_station_per_person,Gas_station_per_junction
0,0.000454,4.666667
1,0.000101,4.000000
2,0.000312,8.000000
3,0.000218,2.000000
4,0.000125,2.000000
5,0.000129,2.000000
6,0.000246,2.000000
7,0.000246,2.250000
8,0.000155,0.500000
9,0.000274,4.333333


Next we have to normalize the data before modeling. Our data is with different magnitudes and this could result an incorect clustering results. We use StandardScaler() to normalize our dataset.

In [27]:
from sklearn.preprocessing import StandardScaler

X = Vilnius_data_clustering.values
X = np.nan_to_num(X)
cluster_dataset = StandardScaler().fit_transform(X)
cluster_dataset

array([[ 1.8090387 ,  1.22378   ],
       [-0.78393937,  0.85213315],
       [ 0.76495406,  3.08201428],
       [ 0.07966   , -0.26280742],
       [-0.60565621, -0.26280742],
       [-0.57497708, -0.26280742],
       [ 0.28416327, -0.26280742],
       [ 0.28211304, -0.12343985],
       [-0.38349842, -1.09901284],
       [ 0.49023016,  1.03795657],
       [-1.05288738, -0.26280742],
       [ 2.09236865,  0.29466286],
       [-0.89180371, -0.8202777 ],
       [-0.61056816,  0.85213315],
       [-0.63367728, -1.09901284],
       [-0.99741312, -1.15475987],
       [-0.82526202, -0.8202777 ],
       [-1.3000429 , -0.8202777 ],
       [ 1.77424352,  0.85213315],
       [-0.20671962, -0.26280742],
       [ 1.2896739 , -0.68091013]])

Now when we have normalized data. We can run clustering model - *kmeans* in our case. 

Problem is that we don't know clusters number. To find the best *k* in *kmeans* we will run for loop and evaluate ech model for diferent *ks'*. We will calculate The Silhouette Coefficient.

We have chosen this index because the ground truth labels are not known and evaluation must be performed using the model itself. 

The Silhouette Coefficient (sklearn.metrics.silhouette_score) is an example of such an evaluation, where a higher Silhouette Coefficient score relates to a model with better defined clusters. The Silhouette Coefficient is defined for each sample and is composed of two scores:

    a. The mean distance between a sample and all other points in the same class.
    b. The mean distance between a sample and all other points in the next nearest cluster.

In [59]:
kclusters=[1,2,3,4,5,6,7,8,9,10]
N=len(kclusters)
for n in range(N):
    i=n+2
    kmeans = KMeans(n_clusters=i, random_state=0).fit(cluster_dataset)
    print('Clusters number: ', i)
    print(kmeans.labels_)
    print(metrics.silhouette_score(cluster_dataset, kmeans.labels_, metric='euclidean'))

Clusters number:  2
[0 1 0 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1]
0.5313096074829202
Clusters number:  3
[2 1 1 0 0 0 0 0 0 1 0 2 0 1 0 0 0 0 2 0 2]
0.48932450056496973
Clusters number:  4
[1 2 3 2 2 2 2 2 0 1 0 1 0 2 0 0 0 0 1 2 1]
0.3659780203094309
Clusters number:  5
[1 4 3 2 0 0 2 2 0 4 0 1 0 4 0 0 0 0 1 2 2]
0.4757765382385509
Clusters number:  6
[1 3 5 2 2 2 2 2 0 3 0 1 0 3 0 0 0 0 1 2 4]
0.4242908734332278
Clusters number:  7
[1 0 3 4 2 2 4 4 5 0 2 1 5 0 5 5 5 5 1 4 6]
0.43703696562544847
Clusters number:  8
[1 4 3 2 7 7 2 2 0 6 7 1 0 4 0 0 0 0 1 2 5]
0.46370369402441963
Clusters number:  9
[1 4 3 0 7 7 0 0 2 5 7 8 2 4 2 2 2 2 1 0 6]
0.443163391611505
Clusters number:  10
[1 6 4 2 5 5 2 2 0 3 5 8 9 6 0 9 9 9 1 2 7]
0.4343105545937422
Clusters number:  11
[ 1  0  4  3 10 10  3  3  2  6  5  9  8  0  2  8  8  8  1 10  7]
0.39508456881875353


As we can see the best score we get when clusters number is set to 2. But we get similar results with using clusters number 3, 5 and 8. Because our purpose is to narrow down the neighborhoods number we will choose higher clusters number - 8.

So we will continue anlysis with cluster number 8.

In [60]:
# set number of clusters
kclusters = 8

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(cluster_dataset)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20]

array([1, 4, 3, 2, 7, 7, 2, 2, 0, 6, 7, 1, 0, 4, 0, 0, 0, 0, 1, 2],
      dtype=int32)

In [63]:
# Calculating the score
print(metrics.silhouette_score(cluster_dataset, kmeans.labels_, metric='euclidean'))

0.46370369402441963


Now we have clustered neighborhoods in Vilnius. We will add cluster labels to our Vilnius neighborhoods dataframe.

In [64]:
# add clustering labels
Vilnius_data_merged.insert(0, 'Cluster Labels', kmeans.labels_)

Vilnius_data_merged

,Cluster Labels,Neighborhoods,Population,Latitude,Longitude,Count_gas_stations,Gas_station_per_person,Count_junctions,Gas_station_per_junction
0,1,Verkiai,30856,54.708707,25.284686,14,0.000454,3,4.666667
1,4,Antakalnis,39697,54.701126,25.308957,4,0.000101,1,4.000000
2,3,Pašilaičiai,25674,54.725942,25.231328,8,0.000312,1,8.000000
3,2,Fabijoniškės,36644,54.723397,25.249529,8,0.000218,4,2.000000
4,7,Pilaitė,15996,54.708126,25.175803,2,0.000125,1,2.000000
5,7,Justiniškės,30958,54.717905,25.220236,4,0.000129,2,2.000000
6,2,Viršuliškės,16250,54.717867,25.220222,4,0.000246,2,2.000000
7,2,Šeškinė,36604,54.715694,25.244574,9,0.000246,4,2.250000
8,0,Šnipiškės,19321,54.692956,25.285007,3,0.000155,6,0.500000
9,6,Žirmūnai,47410,54.723249,25.297213,13,0.000274,3,4.333333


Next we will calculate the average values of each clusters to better understanding of each cluster.

In [65]:
Vilnius_data_merged.groupby('Cluster Labels').mean()

,Population,Latitude,Longitude,Count_gas_stations,Gas_station_per_person,Count_junctions,Gas_station_per_junction
Cluster Labels,,,,,,,
0,22896.000000,54.683866,25.265431,2.000000,0.000093,3.500000,0.733333
1,17317.666667,54.677369,25.213417,8.000000,0.000465,2.000000,3.888889
2,30738.750000,54.704344,25.246685,6.750000,0.000222,3.250000,2.062500
3,25674.000000,54.725942,25.231328,8.000000,0.000312,1.000000,8.000000
4,35930.500000,54.688059,25.255481,4.000000,0.000113,1.000000,4.000000
5,13054.000000,54.677718,25.281702,5.000000,0.000383,4.000000,1.250000
6,47410.000000,54.723249,25.297213,13.000000,0.000274,3.000000,4.333333
7,26043.000000,54.703721,25.200398,2.666667,0.000106,1.333333,2.000000


Now, we can examine each cluster.

In [66]:
Vilnius_data_merged.loc[Vilnius_data_merged['Cluster Labels'] == 0, Vilnius_data_merged.columns[[1] + list(range(5, Vilnius_data_merged.shape[1]))]]

,Neighborhoods,Count_gas_stations,Gas_station_per_person,Count_junctions,Gas_station_per_junction
8,Šnipiškės,3,0.000155,6,0.5
12,Grigiškės,1,0.000086,1,1.0
14,Vilkpėdė,3,0.000121,6,0.5
15,Naujamiestis,2,0.000072,5,0.4
16,Senamiestis,2,0.000095,2,1.0
17,Naujoji Vilnia,1,0.000031,1,1.0


In [67]:
Vilnius_data_merged.loc[Vilnius_data_merged['Cluster Labels'] == 1, Vilnius_data_merged.columns[[1] + list(range(5, Vilnius_data_merged.shape[1]))]]

,Neighborhoods,Count_gas_stations,Gas_station_per_person,Count_junctions,Gas_station_per_junction
0,Verkiai,14,0.000454,3,4.666667
11,Žvėrynas,6,0.000492,2,3.000000
18,Paneriai,4,0.000449,1,4.000000


In [68]:
Vilnius_data_merged.loc[Vilnius_data_merged['Cluster Labels'] == 2, Vilnius_data_merged.columns[[1] + list(range(5, Vilnius_data_merged.shape[1]))]]

,Neighborhoods,Count_gas_stations,Gas_station_per_person,Count_junctions,Gas_station_per_junction
3,Fabijoniškės,8,0.000218,4,2.00
6,Viršuliškės,4,0.000246,2,2.00
7,Šeškinė,9,0.000246,4,2.25
19,Naujininkai,6,0.000179,3,2.00


In [69]:
Vilnius_data_merged.loc[Vilnius_data_merged['Cluster Labels'] == 3, Vilnius_data_merged.columns[[1] + list(range(5, Vilnius_data_merged.shape[1]))]]

,Neighborhoods,Count_gas_stations,Gas_station_per_person,Count_junctions,Gas_station_per_junction
2,Pašilaičiai,8,0.000312,1,8.0


In [70]:
Vilnius_data_merged.loc[Vilnius_data_merged['Cluster Labels'] == 4, Vilnius_data_merged.columns[[1] + list(range(5, Vilnius_data_merged.shape[1]))]]

,Neighborhoods,Count_gas_stations,Gas_station_per_person,Count_junctions,Gas_station_per_junction
1,Antakalnis,4,0.000101,1,4.0
13,Lazdynai,4,0.000124,1,4.0


In [71]:
Vilnius_data_merged.loc[Vilnius_data_merged['Cluster Labels'] == 5, Vilnius_data_merged.columns[[1] + list(range(5, Vilnius_data_merged.shape[1]))]]

,Neighborhoods,Count_gas_stations,Gas_station_per_person,Count_junctions,Gas_station_per_junction
20,Rasos,5,0.000383,4,1.25


In [72]:
Vilnius_data_merged.loc[Vilnius_data_merged['Cluster Labels'] == 6, Vilnius_data_merged.columns[[1] + list(range(5, Vilnius_data_merged.shape[1]))]]

,Neighborhoods,Count_gas_stations,Gas_station_per_person,Count_junctions,Gas_station_per_junction
9,Žirmūnai,13,0.000274,3,4.333333


In [73]:
Vilnius_data_merged.loc[Vilnius_data_merged['Cluster Labels'] == 7, Vilnius_data_merged.columns[[1] + list(range(5, Vilnius_data_merged.shape[1]))]]

,Neighborhoods,Count_gas_stations,Gas_station_per_person,Count_junctions,Gas_station_per_junction
4,Pilaitė,2,0.000125,1,2.0
5,Justiniškės,4,0.000129,2,2.0
10,Karoliniškės,2,0.000064,1,2.0


Finally, let's visualize the resulting clusters

In [77]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Vilnius_data_merged['Latitude'], Vilnius_data_merged['Longitude'], Vilnius_data_merged['Neighborhoods'], Vilnius_data_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

The results show that the best neighborhoods to build new gas station is in cluster 0, where average value of gas stations per person is 0.000093 and average value of gas stations per junction is 0.733333. These average values is lowest.

## Coclusion and insights

As we indicated in our business problem stage company should choose to build a new gas station in place where the traffic volume is the highest and where number of gas stations per resident is the lowest. These neighborhoods  is included in cluster 0.

In [76]:
Vilnius_data_merged.loc[Vilnius_data_merged['Cluster Labels'] == 0, Vilnius_data_merged.columns[[1] + list(range(5, Vilnius_data_merged.shape[1]))]]

,Neighborhoods,Count_gas_stations,Gas_station_per_person,Count_junctions,Gas_station_per_junction
8,Šnipiškės,3,0.000155,6,0.5
12,Grigiškės,1,0.000086,1,1.0
14,Vilkpėdė,3,0.000121,6,0.5
15,Naujamiestis,2,0.000072,5,0.4
16,Senamiestis,2,0.000095,2,1.0
17,Naujoji Vilnia,1,0.000031,1,1.0


As we can see there are 6 neighborhoods in cluster 0: Šnipiškės, Grigiškės, Vilkpedė, Naujamiestis, Senamiestis and Naujoji Vilnia. 

There are some insights from our analysis:

 1. During data collection stage there were 4 neighborhoods with no intevsive traffic junctions and we added these neighborhoods to our dataframe with value 1 (lowest value of traffic) in order to have all neighborhoods in this dataframe. Naujoji Vilnia and Grigiškės were among these 4 neighborhoods. So our offer should be to omit these two neighborhoods, because they did not have any high traffic volume junctions despite these neighborhoods have low gas stations per person number.
 
 2. This leaves us with 4 neighborhoods: Šnipiškės, Vilkpedė, Naujamiestis and Senamiestis. As we can see Naujamiestis has the lowest number of gas stations per person and the lowest number of gas stations per junction. So our first choice should be Naujamiestis neighborhood.
 
 3. Two neighborhoods Šnipiškės and Vilkpedė are also very similar and should be also considered as good choice for new venue.
 
 4. In Senamiestis neighborhood as in an old town of Vilnius there are a lot of restrictions for building anything and a lot of protected heritage. So this neighborhood would not be a good choice despite it has low gas stations per person number and low gas station per junction number.